In [ ]:
import pickle
import json
import re
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
soup = BeautifulSoup(requests.get('https://www.seriouseats.com/').content, 'html.parser')

In [ ]:
links = []
for data in soup.find_all('a', {'class': "global-nav__list-item-link"}):
    if str(data.text).strip() == 'Recipes' or str(data.text).strip() == 'World Cuisines':
        links.append(data.get('href'))   

In [ ]:
get_sublinks = []
for link in links:
    get_data = BeautifulSoup(requests.get(link).content, 'html.parser')
    for sub_data in get_data.find_all('a', {'class': "comp mntl-card-list-items mntl-document-card mntl-card card"}):
        get_sublinks.append(sub_data.get('href'))
links = links+get_sublinks

In [ ]:
links

In [ ]:
recipe = []
for link in links:
    get_data = BeautifulSoup(requests.get(link).content, 'html.parser')
    title = []
    url = []
    for des in get_data.find_all('span', {'class': "card__title"}):
            title.append(str(des.text).strip())
    for link in get_data.find_all('a',{'class':'comp card'}):
           url.append(link['href']) 
    for index, (title, url) in enumerate(zip(title, url)):
        get_datas = BeautifulSoup(requests.get(url).content, 'html.parser')
        try:
            for rate in get_datas.find('div', {'class': "comp aggregate-star-rating__count mntl-aggregate-rating mntl-text-block"}):
                rating = str(rate.text).replace("(" ,"").strip(')').strip()        
        except:
                rating = 0
        serving = 0
        pre_time = 0
        for tim in get_datas.find_all('div', {'class': "loc total-time project-meta__total-time"}):
            pre_time =  tim.find('span', {'class': "meta-text__data"}).text
        for serv in get_datas.find_all('div', {'class': "loc recipe-serving project-meta__recipe-serving"}):
            serving =  serv.find('span', {'class': "meta-text__data"}).text
            serving = [int(word) for word in serving.split() if word.isdigit()][-1]
        ingre = ''
        for inre in get_datas.find_all('ul', {'class': "structured-ingredients__list text-passage"}):
            for val in inre.find_all('li', {'class': "structured-ingredients__list-item"}):
                ingre+=str(val.text).strip()+','
        try:
            nutrition = get_datas.find('table', {'class': 'nutrition-info__table'}).text.split()
            pro = nutrition[-2].replace('g', '')
            carbo = nutrition[-4].replace('g', '')
            fat = nutrition[-6].replace('g', '')
            cal = nutrition[-8]
        except:
            pro = 0
            carbo = 0
            fat = 0
            cal = 0


        recipe.append({'Recipe': title, 'Url': url, 'Ingredients':ingre.rstrip(','), 'Ratings': rating, 'Serving': serving, 
                                       'Prepration_Time': pre_time, 'Calories': cal, 'Protein': pro, 'Carbohydrate': carbo,
                                              'Fat': fat})


In [ ]:
import pandas as pd

df = pd.DataFrame(recipe)
df.to_csv('recipes_set2.csv', index=False)

In [ ]:
df1 = pd.read_csv('recipes_set2.csv')
df1.drop_duplicates(inplace = True)
df1.dropna(subset=['Ingredients'], inplace=True)
df1 = df1.reset_index(drop=True)
df1

In [ ]:
df2 = pd.read_csv('recipes.csv')
df2.drop_duplicates(inplace = True)
df2.dropna(subset=['Ingredients'], inplace=True)
df2 = df2.reset_index(drop=True)
df2

In [ ]:
df3 = df1.append(df2)
df3.drop_duplicates(inplace = True)
df3.to_csv('recipes_final.csv', index=False)
df3

In [ ]:
soup = BeautifulSoup(requests.get('https://www.seriouseats.com/all-recipes-5117985').content, 'html.parser')

links = []
for data in soup.find_all('div', {'class': "comp mntl-taxonomysc-article-list mntl-document-card-list mntl-card-list mntl-block"}):
    links.append(data.find('a', {'class': "truncated-list__item-link"}).get('href'))

In [ ]:
get_sublinks = []
for link in links:
    get_data = BeautifulSoup(requests.get(link).content, 'html.parser')
    for data in get_data.find_all('div', {'class': "truncated-list__item"}):
        get_sublinks.append(data.find('a', {'class': "truncated-list__item-link"}).get('href'))
links = links+get_sublinks

In [ ]:
recipe = []
for link in links:
    get_data = BeautifulSoup(requests.get(link).content, 'html.parser')
    title = []
    url = []
    for des in get_data.find_all('span', {'class': "card__title"}):
            title.append(str(des.text).strip())
    for link in get_data.find_all('a',{'class':'comp card'}):
           url.append(link['href']) 
    for index, (title, url) in enumerate(zip(title, url)):
        get_datas = BeautifulSoup(requests.get(url).content, 'html.parser')
        try:
            for rate in get_datas.find('div', {'class': "comp aggregate-star-rating__count mntl-aggregate-rating mntl-text-block"}):
                rating = str(rate.text).replace("(" ,"").strip(')').strip()        
        except:
                rating = 0
        serving = 0
        pre_time = 0
        for tim in get_datas.find_all('div', {'class': "loc total-time project-meta__total-time"}):
            pre_time =  tim.find('span', {'class': "meta-text__data"}).text
        for serv in get_datas.find_all('div', {'class': "loc recipe-serving project-meta__recipe-serving"}):
            serving =  serv.find('span', {'class': "meta-text__data"}).text
            serving = [int(word) for word in serving.split() if word.isdigit()][-1]
        ingre = ''
        for inre in get_datas.find_all('ul', {'class': "structured-ingredients__list text-passage"}):
            for val in inre.find_all('li', {'class': "structured-ingredients__list-item"}):
                print(val.text)
                ingre+=str(val.text).strip()+','
        try:
            nutrition = get_datas.find('table', {'class': 'nutrition-info__table'}).text.split()
            pro = nutrition[-2].replace('g', '')
            carbo = nutrition[-4].replace('g', '')
            fat = nutrition[-6].replace('g', '')
            cal = nutrition[-8]
        except:
            pro = 0
            carbo = 0
            fat = 0
            cal = 0


        recipe.append({'Recipe': title, 'Url': url, 'Ingredients':ingre.rstrip(','), 'Ratings': rating, 'Serving': serving, 
                                       'Prepration_Time': pre_time, 'Calories': cal, 'Protein': pro, 'Carbohydrate': carbo,
                                              'Fat': fat})

In [ ]:
import pandas as pd

df = pd.DataFrame(recipe)
df.to_csv('recipes.csv', index=False)